<a href="https://colab.research.google.com/github/AdopleAIOrg/Object-Detection/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow
!pip install streamlit
!pip install torch
!pip install transformers
!pip install timm

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw
from typing import Tuple, Union

class ObjectDetection:

    """Class for object detection using the DETR model."""

    def __init__(self):

        """Initialize the object detection class."""

        self.processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101")
        self.model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101")

    def _get_image(self):

      """Get the image from the user."""

      st.title("Object Detection")

      uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
      if uploaded_file is not None:
            image = Image.open(uploaded_file)
            st.image(image, caption="Uploaded Image", use_column_width=True)


            if st.button("Detect Objects"):
                    st.spinner("Processing...")
            return image


    def _process_image(self) -> Tuple[Image.Image, dict]:

        """Process the image and return the results.

        This private method is used internally to process the image using the object detection model.
        It uses the `processor` to prepare the image and the `model` to perform the object detection.
        The results of the detection are then extracted using the `processor`.

        Returns:
            Tuple[Image.Image, dict]: A tuple containing the processed image and the detection results.
                                     The detection results are a dictionary containing keys "scores",
                                     "labels", and "boxes".
        """

        # Get the image from the object detection class (assuming there's a method "_get_image()")
        image = self._get_image()
        # Prepare the image for model input using the "processor" (assuming it's a valid processor)
        inputs = self.processor(images=image, return_tensors="pt")
        # Perform the object detection using the "model" (assuming it's a valid object detection model)
        outputs = self.model(**inputs)

        # Get target sizes to post-process the detection results
        target_sizes = torch.tensor([image.size[::-1]])

        # Post-process the detection outputs to get the final results
        results = self.processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

        # Return the processed image and the detection results
        return image,results

    def show_reuslt(self) -> Union[str, None]:

      """Show the results of the object detection.

      This method processes the image, detects objects using the model, and displays the results.
      If the image processing or object detection fails, it will display an error message.

      Returns:
          str or None: Returns "Upload Image" if processing fails, otherwise returns None.
      """

      try:
          # Process the image and get detection results
          image,results = self._process_image()

          # Loop through the detected objects and draw rectangles around them
          draw = ImageDraw.Draw(image)
          for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
              box = [round(i, 2) for i in box.tolist()]
              draw.rectangle(box, outline="red", width=3)

              # Display the detection results for each object
              st.write(
                  f"Detected {self.model.config.id2label[label.item()]} with confidence "
                  f"{round(score.item(), 3)} at location {box}"
              )

          # Display the image with detected objects
          st.image(image, caption="Detected Objects", use_column_width=True)
      except:
        return "Upload Image"


if __name__ == "__main__":

    object_detec = ObjectDetection()
    object_detec.show_reuslt()

In [10]:
!streamlit run app.py & npx localtunnel --port 8501

  Stopping...
^C
